# TDSP Stage 3 – Modeling

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada **feature engineering RFM**, **transformasi data**, dan **clustering pelanggan** menggunakan Apache Spark.
Hasil dari stage ini adalah segmen pelanggan yang bermakna dan dapat diinterpretasikan secara bisnis.

## 3.1 Tujuan Modeling

Tujuan utama tahap modeling adalah:

1. Mengubah data transaksi menjadi **fitur RFM** pada level pelanggan.
2. Menyiapkan dataset yang siap untuk algoritma clustering.
3. Mengelompokkan pelanggan berdasarkan kemiripan perilaku transaksi.
4. Menghasilkan segmentasi yang **interpretable** dan **actionable** bagi bisnis.

Pendekatan yang digunakan bersifat **unsupervised learning**, karena tidak terdapat label target.

## 3.2 Persiapan Data Modeling

Dataset yang digunakan adalah hasil filter dari TDSP Stage 2.
Langkah awal:
- Pastikan kolom tanggal dalam format timestamp
- Hitung nilai transaksi (Amount)
- Tentukan *reference date* untuk perhitungan Recency

In [ ]:
# ==============================================================
# Import fungsi yang dibutuhkan
# ==============================================================
from pyspark.sql.functions import (
    col, to_timestamp, max as spark_max,
    sum as spark_sum, countDistinct,
    datediff
)


In [ ]:
# ==============================================================
# Pastikan InvoiceDate bertipe timestamp
# ==============================================================
df = df_filtered.withColumn(
    "InvoiceDate",
    to_timestamp(col("InvoiceDate"))
)

df.select("InvoiceDate").printSchema()

## 3.3 Feature Engineering – Monetary

**Monetary** merepresentasikan total nilai uang yang dibelanjakan pelanggan.

Langkah:
- Hitung `Amount = Quantity × Price`
- Agregasi total Amount per pelanggan (PostCode)

In [ ]:
# ==============================================================
# Hitung Amount per baris transaksi
# ==============================================================
df = df.withColumn(
    "Amount",
    col("Quantity") * col("Price")
)

df.select("Quantity", "Price", "Amount").show(5)

## 3.4 Feature Engineering – Recency & Frequency

- **Recency**: jarak waktu (hari) antara transaksi terakhir pelanggan dengan tanggal referensi
- **Frequency**: jumlah transaksi unik (Invoice) per pelanggan


In [ ]:
# ==============================================================
# Tentukan reference date (tanggal transaksi terakhir di dataset)
# ==============================================================
reference_date = df.select(spark_max("InvoiceDate")).collect()[0][0]

reference_date

In [ ]:
# ==============================================================
# Agregasi RFM per pelanggan (PostCode)
# ==============================================================
rfm_df = (
    df.groupBy("PostCode")
      .agg(
          datediff(reference_date, spark_max("InvoiceDate")).alias("Recency"),
          countDistinct("Invoice").alias("Frequency"),
          spark_sum("Amount").alias("Monetary")
      )
)

rfm_df.show(5)

## 3.5 Pemeriksaan Distribusi RFM

Distribusi RFM biasanya **skewed**, khususnya Monetary.
Pemeriksaan statistik deskriptif penting sebelum scaling dan clustering.

In [ ]:
# ==============================================================
# Statistik deskriptif RFM
# ==============================================================
rfm_df.describe().show()

## 3.6 Feature Scaling

Karena perbedaan skala antar fitur RFM cukup besar,
maka diperlukan **normalisasi / scaling** sebelum clustering.

Metode yang digunakan:
- `StandardScaler` (mean = 0, std = 1)

In [ ]:
# ==============================================================
# Vectorization & Scaling
# ==============================================================
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(
    inputCols=["Recency", "Frequency", "Monetary"],
    outputCol="rfm_features"
)

rfm_vector = assembler.transform(rfm_df)

scaler = StandardScaler(
    inputCol="rfm_features",
    outputCol="rfm_scaled",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(rfm_vector)
rfm_scaled_df = scaler_model.transform(rfm_vector)

rfm_scaled_df.select("rfm_scaled").show(5, truncate=False)

## 3.7 Clustering Pelanggan (K-Means)

Algoritma **K-Means** digunakan untuk mengelompokkan pelanggan berdasarkan fitur RFM.

Jumlah cluster ditentukan berdasarkan:
- Interpretabilitas bisnis
- Eksperimen awal

Pada tahap ini digunakan **k = 5**, sesuai praktik umum segmentasi RFM.

In [ ]:
# ==============================================================
# K-Means Clustering
# ==============================================================
from pyspark.ml.clustering import KMeans

kmeans = KMeans(
    featuresCol="rfm_scaled",
    predictionCol="cluster",
    k=5,
    seed=42
)

kmeans_model = kmeans.fit(rfm_scaled_df)
rfm_clustered = kmeans_model.transform(rfm_scaled_df)

rfm_clustered.select("PostCode", "Recency", "Frequency", "Monetary", "cluster").show(10)

## 3.8 Interpretasi Awal Cluster

Langkah awal interpretasi:
- Hitung statistik RFM per cluster
- Bandingkan karakteristik antar cluster

Interpretasi ini akan menjadi dasar rekomendasi bisnis pada stage selanjutnya.

In [ ]:
# ==============================================================
# Statistik RFM per cluster
# ==============================================================
rfm_clustered.groupBy("cluster").agg(
    spark_sum("Monetary").alias("Total_Monetary"),
    spark_max("Monetary").alias("Max_Monetary"),
    spark_max("Frequency").alias("Max_Frequency"),
    spark_max("Recency").alias("Max_Recency"),
    spark_sum("Frequency").alias("Total_Frequency")
).orderBy("cluster").show()

## Ringkasan Stage 3

Pada TDSP Stage 3 ini, kita telah:
- Melakukan feature engineering RFM menggunakan Spark
- Melakukan scaling fitur untuk clustering
- Mengelompokkan pelanggan menggunakan K-Means
- Menghasilkan segmentasi pelanggan awal

Tahap berikutnya adalah **TDSP Stage 4 – Deployment**,
yang berfokus pada penyajian hasil, insight bisnis, dan output yang siap dikonsumsi stakeholder.